<a href="https://colab.research.google.com/github/madacruz/DL4Images/blob/main/Projet_DL4Im.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning for Images

But : réaliser et évaluer un système d'attaque adverse dans le domaine de l'identification faciale, dans un contexte closed-set (càd qu'on suppose qu'il n'y aura pas de nouvelles identités différentes du jeu d'entraînement).

Tâches : 
- Identification faciale : classifier une image parmi un ensemble $n$ de photos (multi-class classification). One-to-many mapping: trouver une personne dans une BD pour l'identifier.
    - Naive approach: softmax loss
    - Attention ce n'est pas de la face verification qu'on souhaite faire = determiner si deux images representent la meme personne ou non (comme en TP). Ou peut etre utiliser face verification pour faire face identification ?
- Attaques adverses



Sous-tâches :
1. Classifier : Entraînement d'un classifier ad-hoc (from scratch) et pre-trained
2. Modèle adverse : l'attaque ne doit pas etre discernable à l'oeil nu
3. Evaluation de la classification et du modèle adverse


Choix du dataset :
LFW (Labeled Faces in the Wild)
- 13 233 images
- 5749 personnes dont 1680 personnes avec plus de 2 photos distinctes 
- Labels = nom des personnes
- Photos centrées sur le visage
- RGB normalisé (pixel entre 0 et 1)
- 250x250 original, 62x47 after preprocessing


Questions :

- Comment préparer le dataset ? Actuellement, on a un ensemble d'images labelisés, donc je peux directement séparer en train, val et test set non ?
    - Comment gérer les personnes qui n'ont qu'une seule image ? On aura peut etre un pb du au fait qu'il y a tres peu d'images pour certaines personnes.
    - Combien de classes ? Il y autant de classes que de personnes, mais du coup le dernier dense layer augmente lineairement avec le nombre de classes.
- Ensuite pour le classifieur : 
    - Baseline : CNN avec softmax output 
    - Un pre-trained 

In [ ]:
from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models

import cv2
from PIL import Image
from pdb import set_trace
import time
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv
import glob
import dlib
import pandas as pd
import numpy as np

## Dataset



In [ ]:
%%capture

!wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
!tar xvzf lfw-deepfunneled.tgz

In [ ]:
PATH = "lfw-deepfunneled/"
USE_SUBSET = True

In [ ]:
dirs = sorted(os.listdir(PATH))
if USE_SUBSET:
    dirs = np.random.choice(dirs, 500, False)

name_to_classid = {d: i for i, d in enumerate(dirs)}
classid_to_name = {v: k for k, v in name_to_classid.items()}
num_classes = len(name_to_classid)

print("Number of classes: ", num_classes)

Number of classes:  500


Dans chaque dossier, il y a une or plusieurs images correspondant à une personne.
- mappings from imagepath and image id: `path_to_id` and `id_to_path`
- mappings from class id to image ids: `classid_to_ids` and `id_to_classid`

In [ ]:
# read all directories
img_paths = {c: [PATH + subfolder + "/" + img
                 for img in sorted(os.listdir(PATH + subfolder))] 
             for subfolder, c in name_to_classid.items()}

# retrieve all images
all_images_path = []
for img_list in img_paths.values():
    all_images_path += img_list

# map to integers
path_to_id = {v: k for k, v in enumerate(all_images_path)} 
id_to_path = {v: k for k, v in path_to_id.items()}

In [ ]:
dict(list(path_to_id.items())[0:13])

{'lfw-deepfunneled/Arnoldo_Aleman/Arnoldo_Aleman_0001.jpg': 1,
 'lfw-deepfunneled/Arnoldo_Aleman/Arnoldo_Aleman_0002.jpg': 2,
 'lfw-deepfunneled/Arnoldo_Aleman/Arnoldo_Aleman_0003.jpg': 3,
 'lfw-deepfunneled/Arnoldo_Aleman/Arnoldo_Aleman_0004.jpg': 4,
 'lfw-deepfunneled/Arnoldo_Aleman/Arnoldo_Aleman_0005.jpg': 5,
 'lfw-deepfunneled/Eli_Stutsman/Eli_Stutsman_0001.jpg': 0,
 'lfw-deepfunneled/Heather_Mills/Heather_Mills_0001.jpg': 6,
 'lfw-deepfunneled/Heather_Mills/Heather_Mills_0002.jpg': 7,
 'lfw-deepfunneled/Heather_Mills/Heather_Mills_0003.jpg': 8,
 'lfw-deepfunneled/Heather_Mills/Heather_Mills_0004.jpg': 9,
 'lfw-deepfunneled/Kevin_Stallings/Kevin_Stallings_0001.jpg': 12,
 'lfw-deepfunneled/Patsy_Hardy/Patsy_Hardy_0001.jpg': 11,
 'lfw-deepfunneled/Paul_Desmarais/Paul_Desmarais_0001.jpg': 10}

In [ ]:
# build mappings between images and class
classid_to_ids = {k: [path_to_id[path] for path in v] for k, v in img_paths.items()}
id_to_classid = {v: c for c,imgs in classid_to_ids.items() for v in imgs}
dict(list(id_to_classid.items())[0:13])

{0: 0,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 2,
 7: 2,
 8: 2,
 9: 2,
 10: 3,
 11: 4,
 12: 5}

Nombre d'images par classe 

In [ ]:
[(classid_to_name[x], len(classid_to_ids[x]))
 for x in np.argsort([len(v) for k,v in classid_to_ids.items()])[::-1][:10]]

[('Colin_Powell', 236),
 ('Gerhard_Schroeder', 109),
 ('Jennifer_Capriati', 42),
 ('Megawati_Sukarnoputri', 33),
 ('Ricardo_Lagos', 27),
 ('Hamid_Karzai', 22),
 ('Carlos_Menem', 21),
 ('John_Bolton', 17),
 ('Norah_Jones', 15),
 ('Kim_Clijsters', 14)]

In [ ]:
#plt.hist([len(v) for k,v in classid_to_ids.items()], bins=range(1,10))
#plt.show()

In [ ]:
# Combien de classes avec 1 ou 2 images ?
# num_images : nb de classes qui ont un nb d'images 'num_images'
from collections import defaultdict

num_classes_with_num_images = defaultdict(int)
for k, v in list(classid_to_ids.items()):
    num_classes_with_num_images[len(v)] += 1

In [ ]:
num_classes_with_num_images

defaultdict(int,
            {1: 342,
             2: 75,
             3: 22,
             4: 18,
             5: 8,
             6: 7,
             7: 6,
             8: 3,
             9: 3,
             10: 1,
             11: 3,
             12: 1,
             13: 1,
             14: 1,
             15: 1,
             17: 1,
             21: 1,
             22: 1,
             27: 1,
             33: 1,
             42: 1,
             109: 1,
             236: 1})


Le choix de la métrique pour évaluer le classifieur est important, puisque si on choisit la mauvaise métrique, alors on choisira un modèle peu performant pour la tâche, ou même être induit en erreur par la valeur de la métrique. 

Choix plus complexe lorsque le jeu de données est déséquilibré. L'accuracy suppose que le jeu de données soit équilibré alors qu'ici, il y a 342 classes qui ont seulement une image, 75 qui en ont deux, etc. 
https://machinelearningmastery.com/what-is-imbalanced-classification/


On peut jouer sur deux variables :
- En agissant sur le jeu de données : upsampling, downsampling ? 
- Comment identifier la abonne métrique pour notre tâche ? Utiliser une autre métrique en plus de l'accuracy, eg ROC AUC (a quel point est-ce que le classifier peut distinguer les classes entre elles) https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/


## Classifieur

La tâche que l'on souhaite résoudre ici est une tâche d'identification (*Face Identification*): étant donné l'image d'un visage d'une personne inconnue, il s'agit d'identifier le nom de cette personne en se basant sur une banque d'images de personnes déjà identifiées.

__Première idée__

Une première manière de formuler ce problème est de le considérer comme une tâche de classification multi-classe. Soit un ensemble de $n$ images avec au total $m$ personnes différentes (donc $m$ classes), il s'agit de classifier une image $x$ parmi ces $m$ personnes. On pourrait donc entraîner un CNN sur le jeu de données de telle sorte que le modèle puisse apprendre à extraire des caractéristiques de chaque visage, puis de les classifier avec un classifieur softmax avec $m$ classes. 
Pour implémenter cette méthode, il faut disposer de beaucoup de données pour chaque classe. Cependant, comme on l'a vu précédemment, le jeu de données est extrêmement déséquilibré (expliquer plus en détail ce problème).

__Deuxième idée__

Une autre manière de résoudre le problème est de partir d'une autre tâche, celle de la vérification de visages (*Face verification*). L'idée ici est de reconnaître si deux images $x_1$ et $x_2$ correspondent ou non à la même personne. Pour cela, on calcule une distance de similarité entre ces deux images, et si la valeur dépasse un certain seuil, alors on considérera que les images représentent la même personne. 


Les modèles siamois permettent de résoudre cette tâche. Deux loss :
- Contrastive loss
- Triplet loss

Avantages des réseaux siamois :
- Fonctionne avec peu de données (expliquer pourquoi, histoire du one-shot learning?)
- Fonctionne avec un jeu déséquilibré (dans notre cas).

Notons que notre tâche est une tâche de classification. Comment donc utiliser 
face verification -> face identification ?

Une requête : soit $x$ une image qu'on souhaite classifier, on construit un ensemble de paires d'images ($x$, $x_2$) où $x2$ est une image qui représente une classe du jeu de données. Il y aura $m$ paires au total, une paire pour chaque classe/personne distincte qui existe dans le jeu. Pour la paire qui donne en sortie 1, on considèrera que $x$ appartient donc à la classe associée à $x_2$. Contrairement à la première idée où on obtient une distribution de probabilités sur les classes (celle avec la plus grande probabilité correspondra à la classe prédite), avec cette méthode, on n'aura pas de probabilités mais une valeur de distance de similarité, et selon cette valeur, on déterminera ou non si l'image appartient à la classe. 

Plusieurs questions :
1. Déjà, que penses-tu de l'idée ?
1. Comment choisir l'image représentative de chaque classe ? 


___

TODO:
1. Baseline : réseau siamois simple (celui qu'on a fait en TP ?)
2. Modèle pre-trained : ArcFace 

___

DeepFace :
- https://scontent-cdt1-1.xx.fbcdn.net/v/t39.8562-6/240890413_887772915161178_4705912772854439762_n.pdf?_nc_cat=109&ccb=1-7&_nc_sid=ad8a9d&_nc_ohc=QcSF_oTtLdAAX-CGlPp&_nc_ht=scontent-cdt1-1.xx&oh=00_AT9NTURwQ1zcn5JMbGswfEJ5THzcTDvzjJ4C0FYOFpb4fA&oe=62BBFCFF
- Rectified input puis archicture : Conv, pool, conv, 3 x locally connected (?), 2 x fully connected, softmax avec k classes -> produit une distribution sur les classes

ArcFace :
- Entrée : 2 images 
- Sortie : Distance entre deux images

https://towardsdatascience.com/finetune-a-facial-recognition-classifier-to-recognize-your-face-using-pytorch-d00a639d9a79

## Modèle adverse

Voir si notre classifieur est facilement attaquable ou non ? Comment le voir ?


## Github